In [1]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
data = pd.read_csv("data/Credit_OTP.csv", encoding='cp1251', sep = ';')

In [3]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('TARGET', axis=1), data['TARGET'], test_size=0.3,
                                                    random_state=42)

In [4]:
data_train = pd.concat([X_train, y_train], axis=1)
data_test = pd.concat([X_test, y_test], axis=1)
data_train.head()

,AGREEMENT_RK,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,EDUCATION,MARITAL_STATUS,GEN_INDUSTRY,...,GEN_PHONE_FL,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_NUM_PAYM,LOAN_DLQ_NUM,LOAN_MAX_DLQ,LOAN_AVG_DLQ_AMT,LOAN_MAX_DLQ_AMT,PREVIOUS_CARD_NUM_UTILIZED,TARGET
51,59916537,37,1,0,0,1,1,Среднее,Состою в браке,Государственная служба,...,1,1,1,5,0,0,0,0,NaN,0
4750,62175061,23,1,0,0,0,0,Неполное среднее,Не состоял в браке,Торговля,...,1,1,0,4,0,0,0,0,NaN,0
2304,61241111,40,1,0,1,1,1,Среднее специальное,Не состоял в браке,Другие сферы,...,1,1,0,5,0,0,0,0,NaN,0
13811,74478393,43,1,0,0,1,1,Высшее,Состою в браке,Транспорт,...,1,1,0,5,0,0,0,0,NaN,0
7717,64428091,56,0,1,1,7,1,Среднее,Не состоял в браке,NaN,...,0,1,0,4,0,0,0,0,NaN,0


In [5]:
data_test.head()

,AGREEMENT_RK,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,EDUCATION,MARITAL_STATUS,GEN_INDUSTRY,...,GEN_PHONE_FL,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_NUM_PAYM,LOAN_DLQ_NUM,LOAN_MAX_DLQ,LOAN_AVG_DLQ_AMT,LOAN_MAX_DLQ_AMT,PREVIOUS_CARD_NUM_UTILIZED,TARGET
6319,63000326,26,1,0,0,0,0,Высшее,Состою в браке,Образование,...,1,1,0,4,0,0,0,0,NaN,0
4252,62024627,59,0,1,0,2,0,Высшее,Состою в браке,NaN,...,0,1,0,5,0,0,0,0,NaN,0
2323,61243333,31,1,0,1,1,1,Среднее,Состою в браке,Ресторанный бизнес/Общественное питание,...,1,1,0,6,0,0,0,0,NaN,0
6896,63635952,24,1,0,1,0,0,Среднее,Состою в браке,Наука,...,1,2,1,17,1,1,1070,1070,NaN,1
3299,61468104,29,1,0,1,2,2,Среднее,Состою в браке,Образование,...,1,1,0,5,0,0,0,0,NaN,0


In [6]:
def small_categories_aggregation(column, n_samples, value='Укрупненная категория', dataset=data):
    '''
    Укрупняет категории и числовые столбцы в датасете
    :param column: имя стоблца в датасете data
    :param n_samples: порог наблюдений, ниже которого категории объединяются в одну
    :param value: значение, которым будет заполнено
    :param dataset: датасет для изменения
    :return:
    '''

    tmp_small_columns = dataset[column].value_counts()[dataset[column].value_counts() < n_samples].index

    for col in tmp_small_columns:
        dataset.at[dataset[column] == col, column] = value
        
def data_processing_version_1(data, dummies=False):
    
    data['PREVIOUS_CARD_NUM_UTILIZED'].fillna(0.0, inplace=True)
    
    data.drop('AGREEMENT_RK', axis=1, inplace=True)
    
    data.drop('DL_DOCUMENT_FL', axis=1, inplace=True)
    
    # imputers
    data['GEN_INDUSTRY'].fillna('Пропуски', inplace=True)
    data['GEN_TITLE'].fillna('Пропуски', inplace=True)
    data['ORG_TP_STATE'].fillna('Пропуски', inplace=True)
    data['ORG_TP_FCAPITAL'].fillna('Пропуски', inplace=True)
    data['JOB_DIR'].fillna('Пропуски', inplace=True)
    data['TP_PROVINCE'].fillna('Пропуски', inplace=True)
    data['REGION_NM'].fillna(data['REGION_NM'].value_counts().index[0], inplace=True)
    
    data.at[data['WORK_TIME'] >= 468, 'WORK_TIME'] = np.NaN
    data['WORK_TIME'].fillna(data['WORK_TIME'].dropna().median(), inplace=True)
    
    for i in ['TARGET', 'SOCSTATUS_WORK_FL', 'SOCSTATUS_PENS_FL', 'GENDER', 
          'EDUCATION', 'MARITAL_STATUS', 'GEN_INDUSTRY', 'GEN_TITLE',
          'ORG_TP_STATE', 'ORG_TP_FCAPITAL', 'JOB_DIR', 'REG_ADDRESS_PROVINCE',
          'FACT_ADDRESS_PROVINCE', 'POSTAL_ADDRESS_PROVINCE', 'TP_PROVINCE', 'REGION_NM',
          'REG_FACT_FL', 'FACT_POST_FL', 'REG_POST_FL', 'REG_FACT_POST_FL', 
          'REG_FACT_POST_TP_FL', 'FL_PRESENCE_FL', 'AUTO_RUS_FL', 'HS_PRESENCE_FL',
          'COT_PRESENCE_FL', 'GAR_PRESENCE_FL', 'LAND_PRESENCE_FL', 'DL_DOCUMENT_FL',
          'GPF_DOCUMENT_FL', 'FACT_PHONE_FL', 'REG_PHONE_FL', 'GEN_PHONE_FL']:
        if i in data.columns:
            data[i]=data[i].astype('str')
        
    for i in ['PERSONAL_INCOME', 'CREDIT', 'FST_PAYMENT', 'LOAN_AVG_DLQ_AMT', 'LOAN_MAX_DLQ_AMT']:
        if i in data.columns:
            data[i] = data[i].str.replace(',', '.').astype('float')

    categorical_columns = [c for c in data.columns if data[c].dtype.name == 'object']
    numerical_columns   = [c for c in data.columns if data[c].dtype.name != 'object']
    
    data.at[data['EDUCATION'] == 'Ученая степень', 'EDUCATION'] = 'Высшее'
    data.at[data['EDUCATION'] == 'Два и более высших образования', 'EDUCATION'] = 'Высшее'
    
    small_categories_aggregation('GEN_INDUSTRY', 100)
    
    data.at[data['GEN_TITLE'] == 'Партнер', 'GEN_TITLE'] = 'Другое'
    data.at[data['ORG_TP_STATE'] == 'Частная ком. с инос. капиталом', 'ORG_TP_STATE'] = 'Частная компания'
    data.at[data['JOB_DIR'] == 'Реклама и маркетинг', 'JOB_DIR'] = 'Реклама и юр. услуги'
    data.at[data['JOB_DIR'] == 'Юридическая служба', 'JOB_DIR'] = 'Реклама и юр. услуги'
    data.at[data['TP_PROVINCE'] == 'Кабардино-Балкария', 'TP_PROVINCE'] = 'Ставропольский край'
    data.at[data['POSTAL_ADDRESS_PROVINCE'] == 'Эвенкийский АО', 'POSTAL_ADDRESS_PROVINCE'] = 'Красноярский край'
    data.at[data['POSTAL_ADDRESS_PROVINCE'] == 'Агинский Бурятский АО', 'POSTAL_ADDRESS_PROVINCE'] = 'Красноярский край'
    data.at[data['POSTAL_ADDRESS_PROVINCE'] == 'Усть-Ордынский Бурятский АО', 'POSTAL_ADDRESS_PROVINCE'] = 'Красноярский край'
    
    data['SOLVENCY'] = data['PERSONAL_INCOME']/(data['CREDIT']/data['TERM'])
    
    if dummies:
        
        data['TARGET'] = data['TARGET'].astype('int')
        data = pd.get_dummies(data)
        return data
    
    else:
        
        for cat in categorical_columns:
            
            tmp_dic = dict.fromkeys(data[cat])
            i = 0
            
            for key in tmp_dic:
                
                tmp_dic[key] = i
                i += 1
            
            data[cat] = data[cat].map(tmp_dic)
            
        for cat in categorical_columns:
            data[cat] = data[cat].astype('str')
        
        return data

In [7]:
# processed_data_train = data_processing_version_1(data_train, True)
# processed_data_test = data_processing_version_1(data_test, True)
# processed_data = pd.concat([processed_data_train, processed_data_test], axis=0)

In [8]:
processed_data = data_processing_version_1(data, True)

In [9]:
processed_data.shape

(15223, 442)

In [10]:
processed_data.head()

,TARGET,AGE,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,OWN_AUTO,CREDIT,TERM,FST_PAYMENT,FACT_LIVING_TERM,...,LAND_PRESENCE_FL_0,LAND_PRESENCE_FL_1,GPF_DOCUMENT_FL_0,GPF_DOCUMENT_FL_1,FACT_PHONE_FL_0,FACT_PHONE_FL_1,REG_PHONE_FL_0,REG_PHONE_FL_1,GEN_PHONE_FL_0,GEN_PHONE_FL_1
0,0,49,2,1,5000.0,0,8000.00,6,8650.0,220,...,1,0,0,1,1,0,1,0,0,1
1,0,32,3,3,12000.0,0,21650.00,6,4000.0,137,...,1,0,0,1,0,1,1,0,0,1
2,0,52,4,0,9000.0,0,33126.00,12,4000.0,251,...,1,0,0,1,1,0,1,0,0,1
3,0,39,1,1,25000.0,0,8491.82,6,5000.0,36,...,1,0,1,0,0,1,0,1,0,1
4,0,30,0,0,12000.0,0,21990.00,12,4000.0,83,...,1,0,0,1,0,1,1,0,0,1


In [11]:
processed_data.to_csv('data/processed_data_version_1.csv', encoding='utf-8')